In [8]:
import sys
sys.path.insert(0, '../funzioni')
from funzioni_ingestion import *

def check_and_scrape_company_pages(df_company_overview, df, file_path):
    old_links = list(df_company_overview['job_link'].unique())
    links = list(df['job_link'].unique())
    new_links = list(set(links) - set(old_links))
    print('Pagine su cui fare scraping', len(new_links))
    df_scraping_company=pd.DataFrame()
    n=5
    for i in range(0, len(new_links), n):
        df_fitt = scraping_company_page(new_links, end_n_page = i+n, start_n_page=i)
        df_scraping_company = df_scraping_company.append(df_fitt).reset_index(drop=True)
        df_company_overview.append(df_scraping_company).reset_index(drop=True).fillna('Non_disponibile').to_csv('../data/company_overview.csv', index=False)
        print('Lista parziale', len(df_scraping_company),'Lista totale', len(pd.read_csv('../data/company_overview.csv')))
    df_final = df.merge(df_scraping_company, how='left', on ='job_link').drop_duplicates().reset_index(drop=True)
    df_final.to_csv(file_path, index=False)

def run_scraping(url=None, file_esclusi=['']):
    import time
    from datetime import date
    path_overview = '../data/company_overview.csv'
    path = '../data'
    today = date.today()

    data = today.strftime("%Y%m%d")
    data_file = today.strftime("%Y-%m-%d")
    
    if url == None:
        print('Ricerco solo schede di valutazione')
        df_company_overview = pd.read_csv(path_overview)
        file_list = os.listdir(path)
        #file_list.remove(['company_overview.csv', 'data_backup'] + file_esclusi)
        file_da_escludere = ['data_backup', 'company_overview.csv', 'jobs']
        file_list = list(set(file_list) - set(file_da_escludere) - set(file_esclusi))
        for e in file_list:
            df_partial_company=pd.DataFrame()
            file_path = path + '/' + e
            df = pd.read_csv(file_path).drop_duplicates().reset_index(drop=True)
            check_and_scrape_company_pages(df_company_overview, df, file_path)
            print('Ho completato il file nel path', file_path)
    else:
        print('Faccio lo scraping del link che mi hai chiesto e poi ricerco le schede di valutazione')
        df_job_scraping = scraping_job_page(url, n_page = 1)
        df_company_overview = pd.read_csv(path_overview)
        df_job_scraping['scraping_date'] = data_file
        file_path=path +"jobs/scraping_job_" + data + '.csv'
        df_job_scraping.to_csv(file_path, index=False)
        check_and_scrape_company_pages(df_company_overview, df_job_scraping, file_path)

In [9]:
run_scraping()

Ricerco solo schede di valutazione
Pagine su cui fare scraping 908
link to scrape:  5
Decline Cookies
check the buttons--> 1
Decline Cookies
check the buttons--> 2
Decline Cookies
check the buttons--> 3
Decline Cookies
check the buttons--> 4
Decline Cookies
check the buttons--> 5
Total pages scraped: 908 

Runtime: 88 seconds
Time sleep:  35 seconds
Lista parziale 0 Lista totale 3129
link to scrape:  5
Decline Cookies
check the buttons--> 1


KeyboardInterrupt: 

In [31]:
df_append = pd.DataFrame()
path = '../data'
file_list = os.listdir(path)
file_da_escludere = ['data_backup', 'company_overview.csv', 'jobs']
file_list = list(set(file_list) - set(file_da_escludere))
for e in file_list:
    file_path = path + '/' + e
    df = pd.read_csv(file_path).drop_duplicates().reset_index(drop=True)
    df_append = df_append.append(df).drop_duplicates().reset_index(drop=True)
df_1 = df_append.groupby(['scraping_date'])[['job_link']].count().reset_index().rename(columns={'job_link':'total_row'})
df_2 = df_append[df_append['oppurtunita_carriera'].isna()].\
groupby(['scraping_date'])[['job_link']].count().reset_index().rename(columns={'job_link':'ko_evaluation'})
df_eval = df_1.merge(df_2, on = 'scraping_date')
df_eval['ratio'] = df_eval['ko_evaluation']/df_eval['total_row']*100
df_eval

,scraping_date,total_row,ko_evaluation,ratio
0,2023-02-16,1386,1386,100.000000
1,2023-02-17,2706,2706,100.000000
2,2023-02-19,900,462,51.333333
3,2023-02-24,900,663,73.666667
4,2023-02-25,900,385,42.777778
5,2023-02-26,360,360,100.000000
6,2023-02-27,869,598,68.814730


In [30]:
# path = '../data'
# dfaaa = pd.read_csv('../data/company_overview.csv')
# file_list = os.listdir(path)
# file_da_escludere = ['data_backup', 'company_overview.csv', 'jobs']
# file_list = list(set(file_list) - set(file_da_escludere))
# for e in file_list:
#     file_path = path + '/' + e
#     df = pd.read_csv(file_path).drop_duplicates().reset_index(drop=True)
#     df = df.iloc[:, 0:8]
#     df_2 = df.merge(dfaaa, how='left', on='job_link')
#     df_2.to_csv(file_path, index=False)